In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# Collab
#%pip install pygam
#%pip install optuna

In [1]:
# %% load packages
import locale
import sys
import os
import time
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import optuna
import requests
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
import random
from sqlalchemy import create_engine,inspect
from pathlib import Path
import urllib.parse
import pyarrow
from calendar import day_abbr
import calendar
from typing import Tuple, Union, Dict, List
from concurrent.futures import ThreadPoolExecutor, as_completed
from pygam import LinearGAM
from datetime import datetime
from typing import List, Dict, Any

/opt/anaconda3/envs/dl-energy-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Collab
#!git clone https://github.com/aamaguay/DLiE_forecast.git
#%cd DLiE_forecast
#!pip install -e .

import random
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [4]:
from srs.utils.tutor_utils import prepare_dataset_tensor, forecasting_study,\
  plot_daily_profile,plot_hour_comparison, build_multiwindow_experts, tune_ewa_eta, \
  ewa_aggregate_forecasts, compute_error_table, tune_expert_window, \
  run_expert_window_test, build_regression_matrix, prepare_train_test_tensors, \
  DST_trafo, prepare_dataset_tensor_modified, build_regression_matrix_modified, \
  build_regression_matrix_modified_2, build_regression_matrix_reordered_dtleak

from srs.utils.our_utils import run_forecast_step
from srs.collect_data.setup import setup_seed, get_device
from srs.collect_data.entsoe_data import create_entsoe_engine, get_tables, get_spec, \
  get_market_divisions,get_map_codes,get_map_codes_starting_with, get_resolution_codes, \
    prepare_generation, prepare_load,prepare_price, prepare_unavailability, \
    prepare_filling_rate_hydro, prepare_physical_flow, prepare_installed_capacity
from srs.collect_data.datastream_data import create_datastream_engine, get_tables, \
  prepare_datastream
from srs.collect_data.dwd_mosmix_data import fetch_region_weather, prepare_weather
from srs.collect_data.merge_data import merge_datasets, build_training_dataset
from srs.models.mlp import SimpleMLP, train_mlp, build_mlp_rolling_forecasts, \
  tune_mlp_hyperparameters, DeepMLP, build_mlp_rolling_forecasts_weighted_loss, build_mlp_rolling_forecasts_weighted_data


In [ ]:
'''
  training interval:
  2019 - 365 days
  2020 - 366 days
  2021 - 365 days
  2022 - 366 days
  
  testing interval:
  2023 - 365 days
  2024 - 366 days
  
  
The reason why I have metnioned lightgbm and GAM before is that I have, as one of the alternative methodologies, to get preliminary predictions from gam,lightbgm or any other models, use these predictions as a input for a input layer to get final predictions from MLP.
That is why I need to be consistent for now with 
'''

### 1. SimpleMLP - rolling window and without Optuna

In [ ]:
# global constants to all zones
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE        = '2023-01-01'
FINAL_DATE_EXPERIMENTS= '2024-12-31'

# hyperparameters
WINDOW_DAYS   = 730                 
HIDDEN_DIM    = 50                  
LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 1e-3
EPOCHS        = 60
BATCH_SIZE    = 32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

repo_root  = Path.cwd().parents[1]
mapcodes   = ["NO1", "NO2", "NO3", "NO4", "NO5"]
zone_data  = {}          

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df_raw   = pd.read_csv(csv_path, parse_dates=["time_utc"])

    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz      = "CET",
        seed    = 42,
        test_days = (pd.Timestamp(FINAL_DATE_EXPERIMENTS)
                     - pd.Timestamp(INIT_TEST_DATE)).days + 1,
        dtype   = torch.float64,
    )

    idx = pd.DatetimeIndex(sorted(train_dates))
    start_i = idx.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    end_i   = idx.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t  = data_t[start_i:end_i+1]             
    dates_t = pd.Series(train_dates[start_i:end_i+1])

    zone_data[code] = dict(
        df        = df_raw,
        tensor    = data_t,
        dates     = dates_t,
        price_t   = price_t[start_i:end_i+1],
    )

# rolling-window MLP per zone
rmse_mlp_by_zone   = {}
preds_mlp_by_zone  = {}

for code in mapcodes:
    print(f"\n==== Zone {code} ====")

    reg_data = build_regression_matrix(
        dat_eval = zone_data[code]["tensor"].cpu().numpy(),
        days_eval= pd.to_datetime(zone_data[code]["dates"]),
        reg_names= zone_data[code]["df"].columns[1:],   
    )
    reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
    dep_idx  = reg_data["dep_indices"]

    all_dates = pd.DatetimeIndex(sorted(zone_data[code]["dates"]\
                                        .iloc[len(zone_data[code]["dates"])
                                             - len(reg_df):]))
    test_start_row = all_dates.get_loc(pd.Timestamp(INIT_TEST_DATE))
    test_end_row   = all_dates.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    horizon        = test_end_row - test_start_row + 1      

    preds, trues = build_mlp_rolling_forecasts(
        regmat_df   = reg_df.astype("float32"),
        dep_indices = dep_idx,
        window      = WINDOW_DAYS,
        horizon     = horizon,
        start_row   = test_start_row,
        hidden_dim  = HIDDEN_DIM,
        lr          = LEARNING_RATE,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
        epochs      = EPOCHS,
        device      = device,
    )

    rmse = torch.sqrt(((preds - trues) ** 2).mean()).item()
    rmse_mlp_by_zone[code]  = rmse
    preds_mlp_by_zone[code] = preds                   

    print(f"RMSE 2023-24: {rmse:7.3f}")

print("\n===== Rolling-MLP RMSE ( NOK / MWh ) =====")
for z, r in rmse_mlp_by_zone.items():
    print(f"{z}:  {r:7.3f}")


In [ ]:
# -------------------------------------------------------------
# 3)  Append to forecast_all and compute error table
mlp_chan  = preds_mlp.unsqueeze(2)            # (N,24,1)
forecast_all = torch.cat([forecast_all, mlp_chan], dim=2)
model_names  = model_names + ["MLP"]

err_table_with_mlp = compute_error_table(forecast_all, model_names)
print(err_table_with_mlp)


Rolling MLP RMSE 2023-24: \
NO1 21.057 \
NO2 24.377 \
NO3 15.700 \
NO4 11.912 \
NO5 17.403

### 2. Deep MLP – Rolling‑window

| **Component**          | **Sub-Component**         | **Details**                                                                                                                                                                                                           |
| ---------------------- | ------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Features**           | **Per-hour block (×24)**  | • `Price_s{h}`: current-day spot price (target)<br> • `Price_lag_{1,2,7}_s{h}`: lags 1,2,7 days<br> • `{Load,Solar,WindOn,WindOff}_DA_lag_0_s{h}`: day-ahead quantities (no lag)<br> **⇒ 8 cols/hour ⇒ 192 total**    |
|                        | **Global block (shared)** | • `WD_{1–7}`: weekday dummies Mon…Sun (7 cols)<br> • `{Coal,NGas,Oil,EUA}_lag_2`: fuel & EUA prices, 2-day lag (4 cols)<br> **⇒ 11 total**                                                                            |
|                        | **Total feature width**   | **192 + 11 = 203**                                                                                                                                                                                                    |
| **Model**              | **DeepMLP**               | Input: 203 ➔ Linear(203→128) ➔ LeakyReLU ➔ Dropout(0.10)<br> ➔ Linear(128→128) ➔ LeakyReLU ➔ Dropout(0.10)<br> ➔ Linear(128→24)  <br>Depth=2 hidden layers<br>Params ≈ (203×128+128)+(128×128+128)+(128×24+24) ≃ 54 k |
| **Training (per day)** | **Pre-processing**        | Standardise X & y with rolling 730-day window (z-score)                                                                                                                                                               |
|                        | **Optimizer**             | Adam(lr=1e-3, weight\_decay=1e-3)                                                                                                                                                                                     |
|                        | **Batch size**            | 64                                                                                                                                                                                                                    |
|                        | **Epochs**                | 60                                                                                                                                                                                                                    |
|                        | **Loss**                  | MSE on the 24-dim output vector                                                                                                                                                                                       |
|                        | **Prediction**            | Forecast next day (predictions per 24h)                                                                                                                                                                                |


In [ ]:
# global constants to all zones
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE        = '2024-01-01'
FINAL_DATE_EXPERIMENTS= '2024-12-31'

# hyperparameters
WINDOW_DAYS   = 730                 
HIDDEN_DIM    = 128 # initial - 50                  
LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 1e-3
EPOCHS        = 60
BATCH_SIZE    = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#%cd /content/DLiE_forecast <- Collab
#repo_root  = Path.cwd().resolve() <-Collab
repo_root  = Path.cwd().parents[1]
mapcodes   = ["NO1", "NO2", "NO3", "NO4", "NO5"]
zone_data  = {}          

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df_raw   = pd.read_csv(csv_path, parse_dates=["time_utc"])

    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz      = "CET",
        seed    = 42,
        test_days = (pd.Timestamp(FINAL_DATE_EXPERIMENTS)
                     - pd.Timestamp(INIT_TEST_DATE)).days + 1,
        dtype   = torch.float64,
    )

    idx = pd.DatetimeIndex(sorted(train_dates))
    start_i = idx.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    end_i   = idx.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t  = data_t[start_i:end_i+1]             
    dates_t = pd.Series(train_dates[start_i:end_i+1])

    zone_data[code] = dict(
        df        = df_raw,
        tensor    = data_t,
        dates     = dates_t,
        price_t   = price_t[start_i:end_i+1],
    )

# rolling-window MLP per zone
rmse_dmlp_by_zone   = {}
preds_dmlp_by_zone  = {}

for code in mapcodes:
    print(f"\n==== Zone {code} ====")

    reg_data = build_regression_matrix(
        dat_eval = zone_data[code]["tensor"].cpu().numpy(),
        days_eval= pd.to_datetime(zone_data[code]["dates"]),
        reg_names= zone_data[code]["df"].columns[1:],   
    )
    reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
    dep_idx  = reg_data["dep_indices"]

    all_dates = pd.DatetimeIndex(sorted(zone_data[code]["dates"]\
                                        .iloc[len(zone_data[code]["dates"])
                                             - len(reg_df):]))
    test_start_row = all_dates.get_loc(pd.Timestamp(INIT_TEST_DATE))
    test_end_row   = all_dates.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    horizon        = test_end_row - test_start_row + 1      

    preds, trues = build_mlp_rolling_forecasts(
        regmat_df   = reg_df.astype("float32"),
        dep_indices = dep_idx,
        window      = WINDOW_DAYS,
        horizon     = horizon,
        start_row   = test_start_row,
        hidden_dim  = HIDDEN_DIM,
        lr          = LEARNING_RATE,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
        epochs      = EPOCHS,
        device      = device,
    )

    rmse_dmlp = torch.sqrt(((preds - trues) ** 2).mean()).item()
    rmse_dmlp_by_zone[code]  = rmse_dmlp
    preds_dmlp_by_zone[code] = preds                   

    print(f"RMSE 2023-24: {rmse_dmlp:7.3f}")

print("\n Rolling-MLP RMSE")
for z, r in rmse_dmlp_by_zone.items():
    print(f"{z}:  {r:7.3f}")


Rolling-MLP RMSE \
NO1:   20.866 \
NO2:   27.100 \
NO3:   13.934 \
NO4:    8.130 \
NO5:   16.446

### 2.1. same as above but traininng interval: 2019.01.01 - 2022.12.31

In [ ]:
# global constants to all zones
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE        = '2023-01-01'
FINAL_DATE_EXPERIMENTS= '2024-12-31'

# hyperparameters
WINDOW_DAYS   = 730                 
HIDDEN_DIM    = 128 # initial - 50                  
LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 1e-3
EPOCHS        = 60
BATCH_SIZE    = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#%cd /content/DLiE_forecast <- Collab
#repo_root  = Path.cwd().resolve() <-Collab
repo_root  = Path.cwd().parents[1]
mapcodes   = ["NO1", "NO2", "NO3", "NO4", "NO5"]
zone_data  = {}          

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df_raw   = pd.read_csv(csv_path, parse_dates=["time_utc"])

    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz      = "CET",
        seed    = 42,
        test_days = (pd.Timestamp(FINAL_DATE_EXPERIMENTS)
                     - pd.Timestamp(INIT_TEST_DATE)).days + 1,
        dtype   = torch.float64,
    )

    idx = pd.DatetimeIndex(sorted(train_dates))
    start_i = idx.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    end_i   = idx.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t  = data_t[start_i:end_i+1]             
    dates_t = pd.Series(train_dates[start_i:end_i+1])

    zone_data[code] = dict(
        df        = df_raw,
        tensor    = data_t,
        dates     = dates_t,
        price_t   = price_t[start_i:end_i+1],
    )

# rolling-window MLP per zone
rmse_dmlp_by_zone   = {}
preds_dmlp_by_zone  = {}

for code in mapcodes:
    print(f"\n==== Zone {code} ====")

    reg_data = build_regression_matrix(
        dat_eval = zone_data[code]["tensor"].cpu().numpy(),
        days_eval= pd.to_datetime(zone_data[code]["dates"]),
        reg_names= zone_data[code]["df"].columns[1:],   
    )
    reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
    dep_idx  = reg_data["dep_indices"]

    all_dates = pd.DatetimeIndex(sorted(zone_data[code]["dates"]\
                                        .iloc[len(zone_data[code]["dates"])
                                             - len(reg_df):]))
    test_start_row = all_dates.get_loc(pd.Timestamp(INIT_TEST_DATE))
    test_end_row   = all_dates.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    horizon        = test_end_row - test_start_row + 1      

    preds, trues = build_mlp_rolling_forecasts(
        regmat_df   = reg_df.astype("float32"),
        dep_indices = dep_idx,
        window      = WINDOW_DAYS,
        horizon     = horizon,
        start_row   = test_start_row,
        hidden_dim  = HIDDEN_DIM,
        lr          = LEARNING_RATE,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
        epochs      = EPOCHS,
        device      = device,
    )

    rmse_dmlp = torch.sqrt(((preds - trues) ** 2).mean()).item()
    rmse_dmlp_by_zone[code]  = rmse_dmlp
    preds_dmlp_by_zone[code] = preds                   

    print(f"RMSE 2023-24: {rmse_dmlp:7.3f}")

print("\n Rolling-MLP RMSE")
for z, r in rmse_dmlp_by_zone.items():
    print(f"{z}:  {r:7.3f}")


Rolling-MLP RMSE \
NO1:   21.212 \
NO2:   26.185 \
NO3:   15.629 \
NO4:   11.252 \
NO5:   17.424

### 2.2. same as above but weight_decay = 0

In [ ]:
# global constants to all zones
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE        = '2023-01-01'
FINAL_DATE_EXPERIMENTS= '2024-12-31'

# global constants
WD                  = [1,2,3,4,5,6,7]
PRICE_LAGS          = [1,2,7]
DA_LAG              = [0]
FUEL_LAGS           = [2]

# hyperparameters
WINDOW_DAYS   = 730                 
HIDDEN_DIM    = 128 # initial - 50                  
LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 0
EPOCHS        = 60
BATCH_SIZE    = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#%cd /content/DLiE_forecast <- Collab
#repo_root  = Path.cwd().resolve() <-Collab
repo_root  = Path.cwd().parents[1]
mapcodes   = ["NO1", "NO2", "NO3", "NO4", "NO5"]
zone_data  = {}          

start_time = time.time()

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df_raw   = pd.read_csv(csv_path, parse_dates=["time_utc"])

    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz      = "CET",
        seed    = 42,
        test_days = (pd.Timestamp(FINAL_DATE_EXPERIMENTS)
                     - pd.Timestamp(INIT_TEST_DATE)).days + 1,
        dtype   = torch.float64,
    )

    idx = pd.DatetimeIndex(sorted(train_dates))
    start_i = idx.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    end_i   = idx.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t  = data_t[start_i:end_i+1]             
    dates_t = pd.Series(train_dates[start_i:end_i+1])

    zone_data[code] = dict(
        df        = df_raw,
        tensor    = data_t,
        dates     = dates_t,
        price_t   = price_t[start_i:end_i+1],
    )

# rolling-window MLP per zone
results_by_zone = {}

for code in mapcodes:
    print(f"\n --- Zone {code} ---")

    reg_data = build_regression_matrix(
        dat_eval = zone_data[code]["tensor"].cpu().numpy(),
        days_eval= pd.to_datetime(zone_data[code]["dates"]),
        reg_names= zone_data[code]["df"].columns[1:],
        wd = WD,
        price_lags = PRICE_LAGS,
        da_lag = DA_LAG,
        fuel_lags = FUEL_LAGS
           
    )
    reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
    dep_idx  = reg_data["dep_indices"]

    all_dates = pd.DatetimeIndex(sorted(zone_data[code]["dates"]\
                                        .iloc[len(zone_data[code]["dates"])
                                             - len(reg_df):]))
    test_start_row = all_dates.get_loc(pd.Timestamp(INIT_TEST_DATE))
    test_end_row   = all_dates.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    horizon        = test_end_row - test_start_row + 1      

    preds, trues, train_rmses, test_rmses = build_mlp_rolling_forecasts(
        regmat_df   = reg_df.astype("float32"),
        dep_indices = dep_idx,
        window      = WINDOW_DAYS,
        horizon     = horizon,
        start_row   = test_start_row,
        hidden_dim  = HIDDEN_DIM,
        lr          = LEARNING_RATE,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
        epochs      = EPOCHS,
        device      = device,
    )

    results_by_zone[code] = {
        "preds":       preds,
        "trues":       trues,
        "train_rmses": train_rmses,
        "test_rmses":   test_rmses,
    }
    
# train vs test rmse summary per region
print("Zone | Train mean | Test mean")
for code, res in results_by_zone.items():
    t = np.array(res["train_rmses"])
    s = np.array(res["test_rmses"])
    print(f"{code} | "
        f"| {t.mean():.3f} | "
        f"| {s.mean():.3f}"  )

end_time = time.time()
duration = (end_time - start_time)/60
print(f"ellapsed time: {duration:.2f} minutes")

# # plot per-day series of test vs train rmse
# for code, res in results_by_zone.items():
#     train = res["train_rmses"]
#     test  = res["test_rmses"]
#     plt.figure()
#     plt.plot(train, label="Train RMSE")
#     plt.plot(test,  label="Test  RMSE")
#     plt.title(f"Zone {code}: RMSE over Forecast Days")
#     plt.xlabel("Forecast Day")
#     plt.ylabel("RMSE")
#     plt.legend()
#     plt.show()
    


In [ ]:
'''
Zone | Train mean | Test mean
NO1 | | 26.435 | | 16.173
NO2 | | 28.034 | | 17.465
NO3 | | 15.615 | | 11.246
NO4 | | 8.936 | | 7.956
NO5 | | 25.369 | | 14.264
ellapsed time: 52.26 minutes
'''

### 2.3 time-decaying weight for MSE loss

We have a classical problem of high bias / under-fit.

2019-2024 includes pre-/post-Ukraine shock, summer-23 reservoir recovery, the autumn-24 price slide.
A rolling window that treats every day equally assumes “one distribution fits all.
1. 2021-22 spike months dominate the MSE loss, so the network wastes capacity trying to be accurate on a pattern that is unlikely to repeat in 2025—but that same focus hurts its fit to the gentler 2024 behaviour that we actually test on. That shows up as under-fitting bias and RMSE that stays stuck in the high teens.
Solution: introduce time-decayed weighting \alpha=0.002 into training

In [ ]:
# global date constants to all zones
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE        = '2023-01-01'
FINAL_DATE_EXPERIMENTS= '2024-12-31'

# global constants
WD                  = [1,2,3,4,5,6,7]
PRICE_LAGS          = [1,2,7] 
DA_LAG              = [0]   # default
FUEL_LAGS           = [2]   # default  

# hyperparameters
WINDOW_DAYS   = 730                 
HIDDEN_DIM    = 128 # initial - 50                  
LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 0
EPOCHS        = 60
BATCH_SIZE    = 64
ALPHA         = 0.002

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#%cd /content/DLiE_forecast <- Collab
#repo_root  = Path.cwd().resolve() <-Collab
repo_root  = Path.cwd().parents[1]
mapcodes   = ["NO1", "NO2", "NO3", "NO4", "NO5"]
zone_data  = {}          

start_time = time.time()

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df_raw   = pd.read_csv(csv_path, parse_dates=["time_utc"])

    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz      = "CET",
        seed    = 42,
        test_days = (pd.Timestamp(FINAL_DATE_EXPERIMENTS)
                     - pd.Timestamp(INIT_TEST_DATE)).days + 1,
        dtype   = torch.float64,
    )

    idx = pd.DatetimeIndex(sorted(train_dates))
    start_i = idx.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    end_i   = idx.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t  = data_t[start_i:end_i+1]             
    dates_t = pd.Series(train_dates[start_i:end_i+1])

    zone_data[code] = dict(
        df        = df_raw,
        tensor    = data_t,
        dates     = dates_t,
        price_t   = price_t[start_i:end_i+1],
    )

# rolling-window MLP per zone
results_by_zone = {}

for code in mapcodes:
    
    reg_data = build_regression_matrix(
        dat_eval = zone_data[code]["tensor"].cpu().numpy(),
        days_eval= pd.to_datetime(zone_data[code]["dates"]),
        reg_names= zone_data[code]["df"].columns[1:],
        wd = WD,
        price_lags = PRICE_LAGS,
        da_lag = DA_LAG,
        fuel_lags = FUEL_LAGS   
    )
    reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
    dep_idx  = reg_data["dep_indices"]

    all_dates = pd.DatetimeIndex(sorted(zone_data[code]["dates"]\
                                        .iloc[len(zone_data[code]["dates"])
                                             - len(reg_df):]))
    test_start_row = all_dates.get_loc(pd.Timestamp(INIT_TEST_DATE))
    test_end_row   = all_dates.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    horizon        = test_end_row - test_start_row + 1      

    preds, trues, train_rmses, test_rmses = build_mlp_rolling_forecasts_weighted_loss(
        regmat_df   = reg_df.astype("float32"),
        dep_indices = dep_idx,
        window      = WINDOW_DAYS,
        horizon     = horizon,
        start_row   = test_start_row,
        hidden_dim  = HIDDEN_DIM,
        lr          = LEARNING_RATE,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
        epochs      = EPOCHS,
        device      = device,
        alpha       = ALPHA
    )

    results_by_zone[code] = {
        "preds":       preds,
        "trues":       trues,
        "train_rmses": train_rmses,
        "test_rmses":   test_rmses,
    }
  
    # train vs test rmse summary per region
    t = np.array(train_rmses)
    s = np.array(test_rmses)
    print(f"--- Zone {code} ---")
    print(f"Train mean RMSE: {t.mean():.3f}")
    print(f"Test mean RMSE: {s.mean():.3f}")

end_time = time.time()
duration = (end_time - start_time)/60
print(f"ellapsed time: {duration:.2f} minutes")

# plot per-day series of test vs train rmse
# for code, res in results_by_zone.items():
#     train = res["train_rmses"]
#     test  = res["test_rmses"]
#     plt.figure()
#     plt.plot(train, label="Train RMSE")
#     plt.plot(test,  label="Test  RMSE")
#     plt.title(f"Zone {code}: RMSE over Forecast Days")
#     plt.xlabel("Forecast Day")
#     plt.ylabel("RMSE")
#     plt.legend()
#     plt.show()
    


In [ ]:
'''--- Zone NO1 ---
Train mean RMSE: 26.326
Test mean RMSE: 16.117
--- Zone NO2 ---
Train mean RMSE: 28.360
Test mean RMSE: 16.987
--- Zone NO3 ---
Train mean RMSE: 16.015
Test mean RMSE: 11.204
--- Zone NO4 ---
Train mean RMSE: 9.458
Test mean RMSE: 8.169
--- Zone NO5 ---
Train mean RMSE: 25.548
Test mean RMSE: 13.598
ellapsed time: 111.12 minutes

### 2.4 time-decaying weight for data (X and y)

In [ ]:
# global date constants to all zones
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE        = '2023-01-01'
FINAL_DATE_EXPERIMENTS= '2024-12-31'

# global constants
WD                  = [1,2,3,4,5,6,7]
PRICE_LAGS          = [1,2,7]
DA_LAG              = [0]
FUEL_LAGS           = [2]

# hyperparameters
WINDOW_DAYS   = 730                 
HIDDEN_DIM    = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 0
EPOCHS        = 60
BATCH_SIZE    = 64
ALPHA         = 0.004 # time-decaying weight for MSE loss or X and y

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#%cd /content/DLiE_forecast <- Collab
#repo_root  = Path.cwd().resolve() <-Collab
repo_root  = Path.cwd().parents[1]
mapcodes   = ["NO1", "NO2", "NO3", "NO4", "NO5"]
zone_data  = {}          

start_time = time.time()

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df_raw   = pd.read_csv(csv_path, parse_dates=["time_utc"])

    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz      = "CET",
        seed    = 42,
        test_days = (pd.Timestamp(FINAL_DATE_EXPERIMENTS)
                     - pd.Timestamp(INIT_TEST_DATE)).days + 1,
        dtype   = torch.float64,
    )

    idx = pd.DatetimeIndex(sorted(train_dates))
    start_i = idx.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    end_i   = idx.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t  = data_t[start_i:end_i+1]             
    dates_t = pd.Series(train_dates[start_i:end_i+1])

    zone_data[code] = dict(
        df        = df_raw,
        tensor    = data_t,
        dates     = dates_t,
        price_t   = price_t[start_i:end_i+1],
    )

# rolling-window MLP per zone
results_by_zone = {}

for code in mapcodes:
    
    reg_data = build_regression_matrix(
        dat_eval = zone_data[code]["tensor"].cpu().numpy(),
        days_eval= pd.to_datetime(zone_data[code]["dates"]),
        reg_names= zone_data[code]["df"].columns[1:],   
        wd = WD,
        price_lags = PRICE_LAGS,
        da_lag = DA_LAG,
        fuel_lags = FUEL_LAGS
    )
    reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
    dep_idx  = reg_data["dep_indices"]

    all_dates = pd.DatetimeIndex(sorted(zone_data[code]["dates"]\
                                        .iloc[len(zone_data[code]["dates"])
                                             - len(reg_df):]))
    test_start_row = all_dates.get_loc(pd.Timestamp(INIT_TEST_DATE))
    test_end_row   = all_dates.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    horizon        = test_end_row - test_start_row + 1      

    preds, trues, train_rmses, test_rmses = build_mlp_rolling_forecasts_weighted_data(
        regmat_df   = reg_df.astype("float32"),
        dep_indices = dep_idx,
        window      = WINDOW_DAYS,
        horizon     = horizon,
        start_row   = test_start_row,
        hidden_dim  = HIDDEN_DIM,
        lr          = LEARNING_RATE,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
        epochs      = EPOCHS,
        alpha       = ALPHA,
        device      = device,
    )

    results_by_zone[code] = {
        "preds":       preds,
        "trues":       trues,
        "train_rmses": train_rmses,
        "test_rmses":   test_rmses,
    }
  
    # train vs test rmse summary per region
    t = np.array(train_rmses)
    s = np.array(test_rmses)
    print(f"--- Zone {code} ---")
    print(f"Train mean RMSE: {t.mean():.3f}")
    print(f"Test mean RMSE: {s.mean():.3f}")

end_time = time.time()
duration = (end_time - start_time)/60
print(f"ellapsed time: {duration:.2f} minutes")

# plot per-day series of test vs train rmse
# for code, res in results_by_zone.items():
#     train = res["train_rmses"]
#     test  = res["test_rmses"]
#     plt.figure()
#     plt.plot(train, label="Train RMSE")
#     plt.plot(test,  label="Test  RMSE")
#     plt.title(f"Zone {code}: RMSE over Forecast Days")
#     plt.xlabel("Forecast Day")
#     plt.ylabel("RMSE")
#     plt.legend()
#     plt.show()
    


In [ ]:
'''
--- Zone NO1 ---
Train mean RMSE: 12.770
Test mean RMSE: 16.757
--- Zone NO2 ---
Train mean RMSE: 13.705
Test mean RMSE: 17.665
--- Zone NO3 ---
Train mean RMSE: 8.095
Test mean RMSE: 11.375
--- Zone NO4 ---
Train mean RMSE: 5.346
Test mean RMSE: 8.149
--- Zone NO5 ---
Train mean RMSE: 12.199
Test mean RMSE: 14.321
ellapsed time: 95.46 minutes
'''

### 2.5 time-decaying weight for data (X and y), ALPHA=0.004, WD=[1,6,7], FUEL_LAGS=[0]

In [ ]:
# global date constants to all zones
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE        = '2023-01-01'
FINAL_DATE_EXPERIMENTS= '2024-12-31'

# global constants
WD                  = [1,6,7]
PRICE_LAGS          = [1,2,7]
DA_LAG              = [0]
FUEL_LAGS           = [0]

# hyperparameters
WINDOW_DAYS   = 730                 
HIDDEN_DIM    = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 0
EPOCHS        = 60
BATCH_SIZE    = 64
ALPHA         = 0.004 # time-decaying weight for MSE loss or X and y

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#%cd /content/DLiE_forecast <- Collab
#repo_root  = Path.cwd().resolve() <-Collab
repo_root  = Path.cwd().parents[1]
mapcodes   = ["NO1", "NO2", "NO3", "NO4", "NO5"]
zone_data  = {}          

start_time = time.time()

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df_raw   = pd.read_csv(csv_path, parse_dates=["time_utc"])

    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz      = "CET",
        seed    = 42,
        test_days = (pd.Timestamp(FINAL_DATE_EXPERIMENTS)
                     - pd.Timestamp(INIT_TEST_DATE)).days + 1,
        dtype   = torch.float64,
    )

    idx = pd.DatetimeIndex(sorted(train_dates))
    start_i = idx.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    end_i   = idx.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t  = data_t[start_i:end_i+1]             
    dates_t = pd.Series(train_dates[start_i:end_i+1])

    zone_data[code] = dict(
        df        = df_raw,
        tensor    = data_t,
        dates     = dates_t,
        price_t   = price_t[start_i:end_i+1],
    )

# rolling-window MLP per zone
results_by_zone = {}

for code in mapcodes:
    
    reg_data = build_regression_matrix(
        dat_eval = zone_data[code]["tensor"].cpu().numpy(),
        days_eval= pd.to_datetime(zone_data[code]["dates"]),
        reg_names= zone_data[code]["df"].columns[1:],   
        wd = WD,
        price_lags = PRICE_LAGS,
        da_lag = DA_LAG,
        fuel_lags = FUEL_LAGS
    )
    reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
    dep_idx  = reg_data["dep_indices"]

    all_dates = pd.DatetimeIndex(sorted(zone_data[code]["dates"]\
                                        .iloc[len(zone_data[code]["dates"])
                                             - len(reg_df):]))
    test_start_row = all_dates.get_loc(pd.Timestamp(INIT_TEST_DATE))
    test_end_row   = all_dates.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    horizon        = test_end_row - test_start_row + 1      

    preds, trues, train_rmses, test_rmses = build_mlp_rolling_forecasts_weighted_data(
        regmat_df   = reg_df.astype("float32"),
        dep_indices = dep_idx,
        window      = WINDOW_DAYS,
        horizon     = horizon,
        start_row   = test_start_row,
        hidden_dim  = HIDDEN_DIM,
        lr          = LEARNING_RATE,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
        epochs      = EPOCHS,
        alpha       = ALPHA,
        device      = device,
    )

    results_by_zone[code] = {
        "preds":       preds,
        "trues":       trues,
        "train_rmses": train_rmses,
        "test_rmses":   test_rmses,
    }
  
    # train vs test rmse summary per region
    t = np.array(train_rmses)
    s = np.array(test_rmses)
    print(f"--- Zone {code} ---")
    print(f"Train mean RMSE: {t.mean():.3f}")
    print(f"Test mean RMSE: {s.mean():.3f}")

end_time = time.time()
duration = (end_time - start_time)/60
print(f"ellapsed time: {duration:.2f} minutes")

# plot per-day series of test vs train rmse
# for code, res in results_by_zone.items():
#     train = res["train_rmses"]
#     test  = res["test_rmses"]
#     plt.figure()
#     plt.plot(train, label="Train RMSE")
#     plt.plot(test,  label="Test  RMSE")
#     plt.title(f"Zone {code}: RMSE over Forecast Days")
#     plt.xlabel("Forecast Day")
#     plt.ylabel("RMSE")
#     plt.legend()
#     plt.show()
    


In [ ]:
'''
--- Zone NO1 ---
Train mean RMSE: 8.264
Test mean RMSE: 17.778
--- Zone NO2 ---
Train mean RMSE: 8.632
Test mean RMSE: 17.980
--- Zone NO3 ---
Train mean RMSE: 5.657
Test mean RMSE: 11.325
--- Zone NO4 ---
Train mean RMSE: 3.898
Test mean RMSE: 8.239
--- Zone NO5 ---
Train mean RMSE: 7.846
Test mean RMSE: 14.480
ellapsed time: 52.87 minutes
'''

### 2.6: WD=[1,2,3,4,5,6,7], FUEL_LAGS=[2], ALPHA=0.004

In [ ]:
# global date constants to all zones
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE        = '2023-01-01'
FINAL_DATE_EXPERIMENTS= '2024-12-31'

# global constants
WD                  = [1,2,3,4,5,6,7]
PRICE_LAGS          = [1,2,7]
DA_LAG              = [0]
FUEL_LAGS           = [2]

# hyperparameters
WINDOW_DAYS   = 730                 
HIDDEN_DIM    = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 0
EPOCHS        = 60
BATCH_SIZE    = 64
ALPHA         = 0.004 # time-decaying weight for MSE loss or X and y

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#%cd /content/DLiE_forecast <- Collab
#repo_root  = Path.cwd().resolve() <-Collab
repo_root  = Path.cwd().parents[1]
mapcodes   = ["NO1", "NO2", "NO3", "NO4", "NO5"]
zone_data  = {}          

start_time = time.time()

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df_raw   = pd.read_csv(csv_path, parse_dates=["time_utc"])

    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz      = "CET",
        seed    = 42,
        test_days = (pd.Timestamp(FINAL_DATE_EXPERIMENTS)
                     - pd.Timestamp(INIT_TEST_DATE)).days + 1,
        dtype   = torch.float64,
    )

    idx = pd.DatetimeIndex(sorted(train_dates))
    start_i = idx.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    end_i   = idx.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t  = data_t[start_i:end_i+1]             
    dates_t = pd.Series(train_dates[start_i:end_i+1])

    zone_data[code] = dict(
        df        = df_raw,
        tensor    = data_t,
        dates     = dates_t,
        price_t   = price_t[start_i:end_i+1],
    )

# rolling-window MLP per zone
results_by_zone = {}

for code in mapcodes:
    
    reg_data = build_regression_matrix(
        dat_eval = zone_data[code]["tensor"].cpu().numpy(),
        days_eval= pd.to_datetime(zone_data[code]["dates"]),
        reg_names= zone_data[code]["df"].columns[1:],   
        wd = WD,
        price_lags = PRICE_LAGS,
        da_lag = DA_LAG,
        fuel_lags = FUEL_LAGS
    )
    reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
    dep_idx  = reg_data["dep_indices"]

    all_dates = pd.DatetimeIndex(sorted(zone_data[code]["dates"]\
                                        .iloc[len(zone_data[code]["dates"])
                                             - len(reg_df):]))
    test_start_row = all_dates.get_loc(pd.Timestamp(INIT_TEST_DATE))
    test_end_row   = all_dates.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    horizon        = test_end_row - test_start_row + 1      

    preds, trues, train_rmses, test_rmses = build_mlp_rolling_forecasts_weighted_data(
        regmat_df   = reg_df.astype("float32"),
        dep_indices = dep_idx,
        window      = WINDOW_DAYS,
        horizon     = horizon,
        start_row   = test_start_row,
        hidden_dim  = HIDDEN_DIM,
        lr          = LEARNING_RATE,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
        epochs      = EPOCHS,
        alpha       = ALPHA,
        device      = device,
    )

    results_by_zone[code] = {
        "preds":       preds,
        "trues":       trues,
        "train_rmses": train_rmses,
        "test_rmses":   test_rmses,
    }
  
    # train vs test rmse summary per region
    t = np.array(train_rmses)
    s = np.array(test_rmses)
    print(f"--- Zone {code} ---")
    print(f"Train mean RMSE: {t.mean():.3f}")
    print(f"Test mean RMSE: {s.mean():.3f}")

end_time = time.time()
duration = (end_time - start_time)/60
print(f"ellapsed time: {duration:.2f} minutes")

# plot per-day series of test vs train rmse
# for code, res in results_by_zone.items():
#     train = res["train_rmses"]
#     test  = res["test_rmses"]
#     plt.figure()
#     plt.plot(train, label="Train RMSE")
#     plt.plot(test,  label="Test  RMSE")
#     plt.title(f"Zone {code}: RMSE over Forecast Days")
#     plt.xlabel("Forecast Day")
#     plt.ylabel("RMSE")
#     plt.legend()
#     plt.show()
    


In [ ]:
'''
--- Zone NO1 ---
Train mean RMSE: 8.088
Test mean RMSE: 17.557
--- Zone NO2 ---
Train mean RMSE: 8.561
Test mean RMSE: 18.480
--- Zone NO3 ---
Train mean RMSE: 5.248
Test mean RMSE: 11.793
--- Zone NO4 ---
Train mean RMSE: 3.741
Test mean RMSE: 8.568
--- Zone NO5 ---
Train mean RMSE: 7.712
Test mean RMSE: 14.659
ellapsed time: 50.17 minutes
'''

### 2.7: WD=[1,6,7], FUEL_LAGS=[0], ALPHA=0.002

In [ ]:
# global date constants to all zones
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE        = '2023-01-01'
FINAL_DATE_EXPERIMENTS= '2024-12-31'

# global constants
WD                  = [1,6,7]
PRICE_LAGS          = [1,2,7]
DA_LAG              = [0]
FUEL_LAGS           = [0]

# hyperparameters
WINDOW_DAYS   = 730                 
HIDDEN_DIM    = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 0
EPOCHS        = 60
BATCH_SIZE    = 64
ALPHA         = 0.002 # time-decaying weight for MSE loss or X and y

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#%cd /content/DLiE_forecast <- Collab
#repo_root  = Path.cwd().resolve() <-Collab
repo_root  = Path.cwd().parents[1]
mapcodes   = ["NO1", "NO2", "NO3", "NO4", "NO5"]
zone_data  = {}          

start_time = time.time()

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df_raw   = pd.read_csv(csv_path, parse_dates=["time_utc"])

    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz      = "CET",
        seed    = 42,
        test_days = (pd.Timestamp(FINAL_DATE_EXPERIMENTS)
                     - pd.Timestamp(INIT_TEST_DATE)).days + 1,
        dtype   = torch.float64,
    )

    idx = pd.DatetimeIndex(sorted(train_dates))
    start_i = idx.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    end_i   = idx.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t  = data_t[start_i:end_i+1]             
    dates_t = pd.Series(train_dates[start_i:end_i+1])

    zone_data[code] = dict(
        df        = df_raw,
        tensor    = data_t,
        dates     = dates_t,
        price_t   = price_t[start_i:end_i+1],
    )

# rolling-window MLP per zone
results_by_zone = {}

for code in mapcodes:
    
    reg_data = build_regression_matrix(
        dat_eval = zone_data[code]["tensor"].cpu().numpy(),
        days_eval= pd.to_datetime(zone_data[code]["dates"]),
        reg_names= zone_data[code]["df"].columns[1:],   
        wd = WD,
        price_lags = PRICE_LAGS,
        da_lag = DA_LAG,
        fuel_lags = FUEL_LAGS
    )
    reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
    dep_idx  = reg_data["dep_indices"]

    all_dates = pd.DatetimeIndex(sorted(zone_data[code]["dates"]\
                                        .iloc[len(zone_data[code]["dates"])
                                             - len(reg_df):]))
    test_start_row = all_dates.get_loc(pd.Timestamp(INIT_TEST_DATE))
    test_end_row   = all_dates.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    horizon        = test_end_row - test_start_row + 1      

    preds, trues, train_rmses, test_rmses = build_mlp_rolling_forecasts_weighted_data(
        regmat_df   = reg_df.astype("float32"),
        dep_indices = dep_idx,
        window      = WINDOW_DAYS,
        horizon     = horizon,
        start_row   = test_start_row,
        hidden_dim  = HIDDEN_DIM,
        lr          = LEARNING_RATE,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
        epochs      = EPOCHS,
        alpha       = ALPHA,
        device      = device,
    )

    results_by_zone[code] = {
        "preds":       preds,
        "trues":       trues,
        "train_rmses": train_rmses,
        "test_rmses":   test_rmses,
    }
  
    # train vs test rmse summary per region
    t = np.array(train_rmses)
    s = np.array(test_rmses)
    print(f"--- Zone {code} ---")
    print(f"Train mean RMSE: {t.mean():.3f}")
    print(f"Test mean RMSE: {s.mean():.3f}")

end_time = time.time()
duration = (end_time - start_time)/60
print(f"ellapsed time: {duration:.2f} minutes")

# plot per-day series of test vs train rmse
# for code, res in results_by_zone.items():
#     train = res["train_rmses"]
#     test  = res["test_rmses"]
#     plt.figure()
#     plt.plot(train, label="Train RMSE")
#     plt.plot(test,  label="Test  RMSE")
#     plt.title(f"Zone {code}: RMSE over Forecast Days")
#     plt.xlabel("Forecast Day")
#     plt.ylabel("RMSE")
#     plt.legend()
#     plt.show()
    


In [ ]:
'''
--- Zone NO1 ---
Train mean RMSE: 12.980
Test mean RMSE: 16.676
--- Zone NO2 ---
Train mean RMSE: 13.861
Test mean RMSE: 17.669
--- Zone NO3 ---
Train mean RMSE: 8.620
Test mean RMSE: 11.331
--- Zone NO4 ---
Train mean RMSE: 5.512
Test mean RMSE: 7.913
--- Zone NO5 ---
Train mean RMSE: 12.446
Test mean RMSE: 14.107
ellapsed time: 874.99 minutes
'''

### 2.8: WD=[1,2,3,4,5,6,7], FUEL_LAGS=[2], ALPHA=0.002

In [ ]:
# global date constants to all zones
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE        = '2023-01-01'
FINAL_DATE_EXPERIMENTS= '2024-12-31'

# global constants
WD                  = [1,2,3,4,5,6,7]
PRICE_LAGS          = [1,2,7]
DA_LAG              = [0]
FUEL_LAGS           = [2]

# hyperparameters
WINDOW_DAYS   = 730                 
HIDDEN_DIM    = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 0
EPOCHS        = 60
BATCH_SIZE    = 64
ALPHA         = 0.002 # time-decaying weight for MSE loss or X and y

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#%cd /content/DLiE_forecast <- Collab
#repo_root  = Path.cwd().resolve() <-Collab
repo_root  = Path.cwd().parents[1]
mapcodes   = ["NO1", "NO2", "NO3", "NO4", "NO5"]
zone_data  = {}          

start_time = time.time()

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df_raw   = pd.read_csv(csv_path, parse_dates=["time_utc"])

    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz      = "CET",
        seed    = 42,
        test_days = (pd.Timestamp(FINAL_DATE_EXPERIMENTS)
                     - pd.Timestamp(INIT_TEST_DATE)).days + 1,
        dtype   = torch.float64,
    )

    idx = pd.DatetimeIndex(sorted(train_dates))
    start_i = idx.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    end_i   = idx.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t  = data_t[start_i:end_i+1]             
    dates_t = pd.Series(train_dates[start_i:end_i+1])

    zone_data[code] = dict(
        df        = df_raw,
        tensor    = data_t,
        dates     = dates_t,
        price_t   = price_t[start_i:end_i+1],
    )

# rolling-window MLP per zone
results_by_zone = {}

for code in mapcodes:
    
    reg_data = build_regression_matrix(
        dat_eval = zone_data[code]["tensor"].cpu().numpy(),
        days_eval= pd.to_datetime(zone_data[code]["dates"]),
        reg_names= zone_data[code]["df"].columns[1:],   
        wd = WD,
        price_lags = PRICE_LAGS,
        da_lag = DA_LAG,
        fuel_lags = FUEL_LAGS
    )
    reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
    dep_idx  = reg_data["dep_indices"]

    all_dates = pd.DatetimeIndex(sorted(zone_data[code]["dates"]\
                                        .iloc[len(zone_data[code]["dates"])
                                             - len(reg_df):]))
    test_start_row = all_dates.get_loc(pd.Timestamp(INIT_TEST_DATE))
    test_end_row   = all_dates.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    horizon        = test_end_row - test_start_row + 1      

    preds, trues, train_rmses, test_rmses = build_mlp_rolling_forecasts_weighted_data(
        regmat_df   = reg_df.astype("float32"),
        dep_indices = dep_idx,
        window      = WINDOW_DAYS,
        horizon     = horizon,
        start_row   = test_start_row,
        hidden_dim  = HIDDEN_DIM,
        lr          = LEARNING_RATE,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
        epochs      = EPOCHS,
        alpha       = ALPHA,
        device      = device,
    )

    results_by_zone[code] = {
        "preds":       preds,
        "trues":       trues,
        "train_rmses": train_rmses,
        "test_rmses":   test_rmses,
    }
  
    # train vs test rmse summary per region
    t = np.array(train_rmses)
    s = np.array(test_rmses)
    print(f"--- Zone {code} ---")
    print(f"Train mean RMSE: {t.mean():.3f}")
    print(f"Test mean RMSE: {s.mean():.3f}")

end_time = time.time()
duration = (end_time - start_time)/60
print(f"ellapsed time: {duration:.2f} minutes")

# plot per-day series of test vs train rmse
# for code, res in results_by_zone.items():
#     train = res["train_rmses"]
#     test  = res["test_rmses"]
#     plt.figure()
#     plt.plot(train, label="Train RMSE")
#     plt.plot(test,  label="Test  RMSE")
#     plt.title(f"Zone {code}: RMSE over Forecast Days")
#     plt.xlabel("Forecast Day")
#     plt.ylabel("RMSE")
#     plt.legend()
#     plt.show()
    


In [ ]:
'''
--- Zone NO1 ---
Train mean RMSE: 12.770
Test mean RMSE: 16.757
--- Zone NO2 ---
Train mean RMSE: 13.705
Test mean RMSE: 17.665
--- Zone NO3 ---
Train mean RMSE: 8.095
Test mean RMSE: 11.375
--- Zone NO4 ---
Train mean RMSE: 5.346
Test mean RMSE: 8.149
--- Zone NO5 ---
Train mean RMSE: 12.199
Test mean RMSE: 14.321
ellapsed time: 63.10 minutes
'''

### 2.9: WD dropped, FUEL_LAGS=[2], ALPHA=0.002

In [ ]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# global date constants to all zones
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE        = '2023-01-01'
FINAL_DATE_EXPERIMENTS= '2024-12-31'

# features
DA_NAMES   = ["Load_DA", "Solar_DA", "WindOn_DA", "WindOff_DA"]
FUEL_NAMES = ["Coal","NGas","Oil","EUA"]

# global constants
WD                  = [0]
PRICE_LAGS          = [1,2,7]
DA_LAG              = [0]
FUEL_LAGS           = [2]

# hyperparameters
WINDOW_DAYS   = 730                 
HIDDEN_DIM    = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 0
EPOCHS        = 60
BATCH_SIZE    = 64
ALPHA         = 0.002 # time-decaying weight for MSE loss or X and y

#%cd /content/DLiE_forecast <- Collab
#repo_root  = Path.cwd().resolve() <-Collab
repo_root  = Path.cwd().parents[1]
mapcodes   = ["NO1", "NO2", "NO3", "NO4", "NO5"]
zone_data  = {}          

start_time = time.time()

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df_raw   = pd.read_csv(csv_path, parse_dates=["time_utc"])

    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz      = "CET",
        seed    = 42,
        test_days = (pd.Timestamp(FINAL_DATE_EXPERIMENTS)
                     - pd.Timestamp(INIT_TEST_DATE)).days + 1,
        dtype   = torch.float64,
    )

    idx = pd.DatetimeIndex(sorted(train_dates))
    start_i = idx.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    end_i   = idx.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t  = data_t[start_i:end_i+1]             
    dates_t = pd.Series(train_dates[start_i:end_i+1])

    zone_data[code] = dict(
        df        = df_raw,
        tensor    = data_t,
        dates     = dates_t,
        price_t   = price_t[start_i:end_i+1],
    )

# rolling-window MLP per zone
results_by_zone = {}

for code in mapcodes:
    
    reg_data = build_regression_matrix(
        dat_eval = zone_data[code]["tensor"].cpu().numpy(),
        days_eval= pd.to_datetime(zone_data[code]["dates"]),
        reg_names= zone_data[code]["df"].columns[1:],   
        wd = WD,
        price_lags = PRICE_LAGS,
        da_lags = DA_LAG,
        fuel_lags = FUEL_LAGS,
        da_names = DA_NAMES,
        fuel_names = FUEL_NAMES
    )
    reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
    dep_idx  = reg_data["dep_indices"]

    all_dates = pd.DatetimeIndex(sorted(zone_data[code]["dates"]\
                                        .iloc[len(zone_data[code]["dates"])
                                             - len(reg_df):]))
    test_start_row = all_dates.get_loc(pd.Timestamp(INIT_TEST_DATE))
    test_end_row   = all_dates.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    horizon        = test_end_row - test_start_row + 1      

    preds, trues, train_rmses, test_rmses = build_mlp_rolling_forecasts_weighted_data(
        regmat_df   = reg_df.astype("float32"),
        dep_indices = dep_idx,
        window      = WINDOW_DAYS,
        horizon     = horizon,
        start_row   = test_start_row,
        hidden_dim  = HIDDEN_DIM,
        lr          = LEARNING_RATE,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
        epochs      = EPOCHS,
        alpha       = ALPHA,
        device      = device,
    )

    results_by_zone[code] = {
        "preds":       preds,
        "trues":       trues,
        "train_rmses": train_rmses,
        "test_rmses":   test_rmses,
    }
  
    # train vs test rmse summary per region
    t = np.array(train_rmses)
    s = np.array(test_rmses)
    print(f"--- Zone {code} ---")
    print(f"Train mean RMSE: {t.mean():.3f}")
    print(f"Test mean RMSE: {s.mean():.3f}")

end_time = time.time()
duration = (end_time - start_time)/60
print(f"ellapsed time: {duration:.2f} minutes")

# plot per-day series of test vs train rmse
# for code, res in results_by_zone.items():
#     train = res["train_rmses"]
#     test  = res["test_rmses"]
#     plt.figure()
#     plt.plot(train, label="Train RMSE")
#     plt.plot(test,  label="Test  RMSE")
#     plt.title(f"Zone {code}: RMSE over Forecast Days")
#     plt.xlabel("Forecast Day")
#     plt.ylabel("RMSE")
#     plt.legend()
#     plt.show()
    


In [71]:
dat_eval = zone_data['NO1']["tensor"].cpu().numpy()
reg_names= zone_data['NO1']["df"].columns[1:]   
days_eval= pd.to_datetime(zone_data["NO1"]["dates"])
# cell N: extract the pieces you want to peek at
n_days, S, n_vars = dat_eval.shape
price_idx = reg_names.get_loc("Price")

# build the Price_s* DataFrame
df_prices = pd.DataFrame({
    f"Price_s{h}": dat_eval[:, h, price_idx]
    for h in range(S)
}, index=days_eval)

# show the first few rows
df_prices["P_daily_mean"] = df_prices.mean(axis=1)
df_prices.head()



,Price_s0,Price_s1,Price_s2,Price_s3,Price_s4,Price_s5,Price_s6,Price_s7,Price_s8,Price_s9,...,Price_s15,Price_s16,Price_s17,Price_s18,Price_s19,Price_s20,Price_s21,Price_s22,Price_s23,P_daily_mean
2019-01-01,NaN,49.25,49.17,48.37,47.19,47.37,48.16,49.09,47.63,47.99,...,48.45,48.98,49.45,48.89,48.91,49.31,49.10,49.42,49.12,48.743478
2019-01-02,49.16,48.14,48.14,47.24,47.53,47.98,49.44,50.11,50.74,50.74,...,52.38,54.67,55.58,55.39,54.90,53.86,52.71,52.09,50.76,50.979167
2019-01-03,50.03,48.89,48.31,48.40,48.79,49.61,51.17,53.91,58.86,59.14,...,66.13,68.41,69.55,65.67,61.51,56.04,54.44,52.09,49.98,57.057500
2019-01-04,49.98,48.67,47.71,47.64,47.43,48.58,50.87,52.32,54.44,54.18,...,51.69,52.59,53.34,52.81,51.79,51.04,50.45,49.79,49.02,51.083333
2019-01-05,48.42,47.52,47.19,47.00,47.01,47.33,47.59,48.22,49.99,50.73,...,53.22,55.26,55.87,55.26,53.80,52.75,51.90,51.13,50.21,50.723750


In [69]:
df_prices["P_daily_mean"] = df_prices.mean(axis=1)
demean_cols = []
for h in range(S):
    center = f"P_center_s{h}"
    df_prices[center] = df_prices[f"Price_s{h}"] - df_prices["P_daily_mean"]
    demean_cols.append(center)
regmat_df = df_prices.drop(columns=[f"Price_s{h}" for h in range(S)])


In [70]:
regmat_df

,P_daily_mean,P_center_s0,P_center_s1,P_center_s2,P_center_s3,P_center_s4,P_center_s5,P_center_s6,P_center_s7,P_center_s8,...,P_center_s14,P_center_s15,P_center_s16,P_center_s17,P_center_s18,P_center_s19,P_center_s20,P_center_s21,P_center_s22,P_center_s23
2019-01-01,48.743478,NaN,0.506522,0.426522,-0.373478,-1.553478,-1.373478,-0.583478,0.346522,-1.113478,...,-0.413478,-0.293478,0.236522,0.706522,0.146522,0.166522,0.566522,0.356522,0.676522,0.376522
2019-01-02,50.979167,-1.819167,-2.839167,-2.839167,-3.739167,-3.449167,-2.999167,-1.539167,-0.869167,-0.239167,...,-0.049167,1.400833,3.690833,4.600833,4.410833,3.920833,2.880833,1.730833,1.110833,-0.219167
2019-01-03,57.057500,-7.027500,-8.167500,-8.747500,-8.657500,-8.267500,-7.447500,-5.887500,-3.147500,1.802500,...,6.612500,9.072500,11.352500,12.492500,8.612500,4.452500,-1.017500,-2.617500,-4.967500,-7.077500
2019-01-04,51.083333,-1.103333,-2.413333,-3.373333,-3.443333,-3.653333,-2.503333,-0.213333,1.236667,3.356667,...,0.326667,0.606667,1.506667,2.256667,1.726667,0.706667,-0.043333,-0.633333,-1.293333,-2.063333
2019-01-05,50.723750,-2.303750,-3.203750,-3.533750,-3.723750,-3.713750,-3.393750,-3.133750,-2.503750,-0.733750,...,1.196250,2.496250,4.536250,5.146250,4.536250,3.076250,2.026250,1.176250,0.406250,-0.513750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27,32.546667,-6.806667,-7.246667,-7.236667,-10.246667,-13.186667,-3.196667,-1.026667,1.523333,4.083333,...,4.893333,5.123333,5.283333,5.403333,3.583333,1.853333,0.843333,-0.956667,-1.486667,-3.476667
2024-12-28,28.117917,1.042083,-1.337917,-3.347917,-4.457917,-4.197917,-4.537917,-3.377917,-1.347917,1.562083,...,3.272083,3.472083,3.552083,3.502083,3.202083,-0.197917,-2.717917,-3.077917,-3.627917,-4.697917
2024-12-29,23.936250,4.223750,-5.576250,-5.736250,-5.946250,-5.626250,-7.486250,-7.216250,-3.226250,3.763750,...,2.033750,1.983750,3.393750,4.003750,2.403750,1.593750,0.813750,0.433750,-0.396250,-1.416250
2024-12-30,31.408333,-4.408333,-5.868333,-6.998333,-4.868333,-5.998333,-4.048333,-2.258333,-1.158333,0.421667,...,2.231667,3.571667,5.631667,5.111667,2.941667,-0.078333,0.241667,0.501667,0.591667,0.541667


In [ ]:
'''
--- Zone NO1 ---
Train mean RMSE: 13.592
Test mean RMSE: 17.289
--- Zone NO2 ---
Train mean RMSE: 14.706
Test mean RMSE: 17.846
--- Zone NO3 ---
Train mean RMSE: 9.103
Test mean RMSE: 11.291
--- Zone NO4 ---
Train mean RMSE: 5.825
Test mean RMSE: 7.927
--- Zone NO5 ---
Train mean RMSE: 12.799
Test mean RMSE: 14.017
ellapsed time: 52.56 minutes
'''

### 3: De-meaned Price

In [6]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# global date constants to all zones
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE        = '2023-01-01'
FINAL_DATE_EXPERIMENTS= '2024-12-31'

# features
DA_NAMES   = ["Load_DA", "Solar_DA", "WindOn_DA", "WindOff_DA"]
FUEL_NAMES = ["Coal","NGas","Oil","EUA"]

# global constants
WD                  = [0]
PRICE_LAGS          = [1,2,7]
DA_LAG              = [0]
FUEL_LAGS           = [2]

# hyperparameters
WINDOW_DAYS   = 730                 
HIDDEN_DIM    = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 0
EPOCHS        = 60
BATCH_SIZE    = 64
ALPHA         = 0.002 # time-decaying weight for MSE loss or X and y

#%cd /content/DLiE_forecast <- Collab
#repo_root  = Path.cwd().resolve() <-Collab
repo_root  = Path.cwd().parents[1]
mapcodes   = ["NO1", "NO2", "NO3", "NO4", "NO5"]
zone_data  = {}          

start_time = time.time()

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df_raw   = pd.read_csv(csv_path, parse_dates=["time_utc"])

    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz      = "CET",
        seed    = 42,
        test_days = (pd.Timestamp(FINAL_DATE_EXPERIMENTS)
                     - pd.Timestamp(INIT_TEST_DATE)).days + 1,
        dtype   = torch.float64,
    )

    idx = pd.DatetimeIndex(sorted(train_dates))
    start_i = idx.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    end_i   = idx.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t  = data_t[start_i:end_i+1]             
    dates_t = pd.Series(train_dates[start_i:end_i+1])

    zone_data[code] = dict(
        df        = df_raw,
        tensor    = data_t,
        dates     = dates_t,
        price_t   = price_t[start_i:end_i+1],
    )

# rolling-window MLP per zone
results_by_zone = {}

for code in mapcodes:
    
    reg_data = build_regression_matrix_modified(
        dat_eval = zone_data[code]["tensor"].cpu().numpy(),
        days_eval= pd.to_datetime(zone_data[code]["dates"]),
        reg_names= zone_data[code]["df"].columns[1:],   
        wd = WD,
        price_lags = PRICE_LAGS,
        da_lag = DA_LAG,
        fuel_lags = FUEL_LAGS,
        da_names = DA_NAMES,
        fuel_names = FUEL_NAMES
    )
    reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
    dep_idx  = reg_data["dep_indices"]

    all_dates = pd.DatetimeIndex(sorted(zone_data[code]["dates"]\
                                        .iloc[len(zone_data[code]["dates"])
                                             - len(reg_df):]))
    test_start_row = all_dates.get_loc(pd.Timestamp(INIT_TEST_DATE))
    test_end_row   = all_dates.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    horizon        = test_end_row - test_start_row + 1      

    preds, trues, train_rmses, test_rmses = build_mlp_rolling_forecasts_weighted_data(
        regmat_df   = reg_df.astype("float32"),
        dep_indices = dep_idx,
        window      = WINDOW_DAYS,
        horizon     = horizon,
        start_row   = test_start_row,
        hidden_dim  = HIDDEN_DIM,
        lr          = LEARNING_RATE,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
        epochs      = EPOCHS,
        alpha       = ALPHA,
        device      = device,
    )

    results_by_zone[code] = {
        "preds":       preds,
        "trues":       trues,
        "train_rmses": train_rmses,
        "test_rmses":   test_rmses,
    }
  
    # train vs test rmse summary per region
    t = np.array(train_rmses)
    s = np.array(test_rmses)
    print(f"--- Zone {code} ---")
    print(f"Train mean RMSE: {t.mean():.3f}")
    print(f"Test mean RMSE: {s.mean():.3f}")

end_time = time.time()
duration = (end_time - start_time)/60
print(f"ellapsed time: {duration:.2f} minutes")

--- Zone NO1 ---
Train mean RMSE: 7.658
Test mean RMSE: 9.632
--- Zone NO2 ---
Train mean RMSE: 8.322
Test mean RMSE: 11.174
--- Zone NO3 ---
Train mean RMSE: 4.550
Test mean RMSE: 6.201
--- Zone NO4 ---
Train mean RMSE: 3.111
Test mean RMSE: 4.487
--- Zone NO5 ---
Train mean RMSE: 7.145
Test mean RMSE: 7.899
ellapsed time: 771.51 minutes


In [ ]:
'''
--- Zone NO1 ---
Train mean RMSE: 7.611
Test mean RMSE: 9.589
--- Zone NO2 ---
Train mean RMSE: 8.263
Test mean RMSE: 11.110
--- Zone NO3 ---
Train mean RMSE: 4.506
Test mean RMSE: 6.207
--- Zone NO4 ---
Train mean RMSE: 3.067
Test mean RMSE: 4.543
--- Zone NO5 ---
Train mean RMSE: 7.094
Test mean RMSE: 7.776
ellapsed time: 51.79 minutes'''

### 3.1: Dropped: WindOff, Solar_DA, 

In [12]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# global date constants to all zones
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE        = '2023-01-01'
FINAL_DATE_EXPERIMENTS= '2024-12-31'

# features
DA_NAMES   = ["Load_DA", "WindOn_DA"]
FUEL_NAMES = ["Coal","NGas","Oil","EUA"]

# global constants
WD                  = [0]
PRICE_LAGS          = [1,2,7]
DA_LAG              = [0]
FUEL_LAGS           = [2]

# hyperparameters
WINDOW_DAYS   = 730                 
HIDDEN_DIM    = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 0
EPOCHS        = 60
BATCH_SIZE    = 64
ALPHA         = 0.002 # time-decaying weight for MSE loss or X and y

#%cd /content/DLiE_forecast <- Collab
#repo_root  = Path.cwd().resolve() <-Collab
repo_root  = Path.cwd().parents[1]
mapcodes   = ["NO1", "NO2", "NO3", "NO4", "NO5"]
zone_data  = {}          

start_time = time.time()

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df_raw   = pd.read_csv(csv_path, parse_dates=["time_utc"])

    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz      = "CET",
        seed    = 42,
        test_days = (pd.Timestamp(FINAL_DATE_EXPERIMENTS)
                     - pd.Timestamp(INIT_TEST_DATE)).days + 1,
        dtype   = torch.float64,
    )

    idx = pd.DatetimeIndex(sorted(train_dates))
    start_i = idx.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    end_i   = idx.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t  = data_t[start_i:end_i+1]             
    dates_t = pd.Series(train_dates[start_i:end_i+1])

    zone_data[code] = dict(
        df        = df_raw,
        tensor    = data_t,
        dates     = dates_t,
        price_t   = price_t[start_i:end_i+1],
    )

# rolling-window MLP per zone
results_by_zone = {}

for code in mapcodes:
    
    reg_data = build_regression_matrix_modified(
        dat_eval = zone_data[code]["tensor"].cpu().numpy(),
        days_eval= pd.to_datetime(zone_data[code]["dates"]),
        reg_names= zone_data[code]["df"].columns[1:],   
        wd = WD,
        price_lags = PRICE_LAGS,
        da_lag = DA_LAG,
        fuel_lags = FUEL_LAGS,
        da_names = DA_NAMES,
        fuel_names = FUEL_NAMES
    )
    reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
    dep_idx  = reg_data["dep_indices"]

    all_dates = pd.DatetimeIndex(sorted(zone_data[code]["dates"]\
                                        .iloc[len(zone_data[code]["dates"])
                                             - len(reg_df):]))
    test_start_row = all_dates.get_loc(pd.Timestamp(INIT_TEST_DATE))
    test_end_row   = all_dates.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    horizon        = test_end_row - test_start_row + 1      

    preds, trues, train_rmses, test_rmses = build_mlp_rolling_forecasts_weighted_data(
        regmat_df   = reg_df.astype("float32"),
        dep_indices = dep_idx,
        window      = WINDOW_DAYS,
        horizon     = horizon,
        start_row   = test_start_row,
        hidden_dim  = HIDDEN_DIM,
        lr          = LEARNING_RATE,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
        epochs      = EPOCHS,
        alpha       = ALPHA,
        device      = device,
    )

    results_by_zone[code] = {
        "preds":       preds,
        "trues":       trues,
        "train_rmses": train_rmses,
        "test_rmses":   test_rmses,
    }
  
    # train vs test rmse summary per region
    t = np.array(train_rmses)
    s = np.array(test_rmses)
    print(f"--- Zone {code} ---")
    print(f"Train mean RMSE: {t.mean():.3f}")
    print(f"Test mean RMSE: {s.mean():.3f}")

end_time = time.time()
duration = (end_time - start_time)/60
print(f"ellapsed time: {duration:.2f} minutes")

--- Zone NO1 ---
Train mean RMSE: 7.611
Test mean RMSE: 9.589
--- Zone NO2 ---
Train mean RMSE: 8.263
Test mean RMSE: 11.110
--- Zone NO3 ---
Train mean RMSE: 4.506
Test mean RMSE: 6.207
--- Zone NO4 ---
Train mean RMSE: 3.067
Test mean RMSE: 4.543
--- Zone NO5 ---
Train mean RMSE: 7.094
Test mean RMSE: 7.776
ellapsed time: 51.79 minutes


### 3.2 Same as above but MLP is trained using build_mlp_rolling_forecasts()

In [ ]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# global date constants to all zones
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE        = '2023-01-01'
FINAL_DATE_EXPERIMENTS= '2024-12-31'

# features
DA_NAMES   = ["Load_DA", "WindOn_DA"]
FUEL_NAMES = ["Coal","NGas","Oil","EUA"]

# global constants
WD                  = [0]
PRICE_LAGS          = [1,2,7]
DA_LAG              = [0]
FUEL_LAGS           = [2]

# hyperparameters
WINDOW_DAYS   = 730                 
HIDDEN_DIM    = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 0
EPOCHS        = 60
BATCH_SIZE    = 64
ALPHA         = 0.002 # time-decaying weight for MSE loss or X and y

#%cd /content/DLiE_forecast <- Collab
#repo_root  = Path.cwd().resolve() <-Collab
repo_root  = Path.cwd().parents[1]
mapcodes   = ["NO1", "NO2", "NO3", "NO4", "NO5"]
zone_data  = {}          

start_time = time.time()

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df_raw   = pd.read_csv(csv_path, parse_dates=["time_utc"])

    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz      = "CET",
        seed    = 42,
        test_days = (pd.Timestamp(FINAL_DATE_EXPERIMENTS)
                     - pd.Timestamp(INIT_TEST_DATE)).days + 1,
        dtype   = torch.float64,
    )

    idx = pd.DatetimeIndex(sorted(train_dates))
    start_i = idx.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    end_i   = idx.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t  = data_t[start_i:end_i+1]             
    dates_t = pd.Series(train_dates[start_i:end_i+1])

    zone_data[code] = dict(
        df        = df_raw,
        tensor    = data_t,
        dates     = dates_t,
        price_t   = price_t[start_i:end_i+1],
    )

# rolling-window MLP per zone
results_by_zone = {}

for code in mapcodes:
    
    reg_data = build_regression_matrix_modified(
        dat_eval = zone_data[code]["tensor"].cpu().numpy(),
        days_eval= pd.to_datetime(zone_data[code]["dates"]),
        reg_names= zone_data[code]["df"].columns[1:],   
        wd = WD,
        price_lags = PRICE_LAGS,
        da_lag = DA_LAG,
        fuel_lags = FUEL_LAGS,
        da_names = DA_NAMES,
        fuel_names = FUEL_NAMES
    )
    reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
    dep_idx  = reg_data["dep_indices"]

    all_dates = pd.DatetimeIndex(sorted(zone_data[code]["dates"]\
                                        .iloc[len(zone_data[code]["dates"])
                                             - len(reg_df):]))
    test_start_row = all_dates.get_loc(pd.Timestamp(INIT_TEST_DATE))
    test_end_row   = all_dates.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    horizon        = test_end_row - test_start_row + 1      

    preds, trues, train_rmses, test_rmses = build_mlp_rolling_forecasts(
        regmat_df   = reg_df.astype("float32"),
        dep_indices = dep_idx,
        window      = WINDOW_DAYS,
        horizon     = horizon,
        start_row   = test_start_row,
        hidden_dim  = HIDDEN_DIM,
        lr          = LEARNING_RATE,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
        epochs      = EPOCHS,
        device      = device,
    )

    results_by_zone[code] = {
        "preds":       preds,
        "trues":       trues,
        "train_rmses": train_rmses,
        "test_rmses":   test_rmses,
    }
  
    # train vs test rmse summary per region
    t = np.array(train_rmses)
    s = np.array(test_rmses)
    print(f"--- Zone {code} ---")
    print(f"Train mean RMSE: {t.mean():.3f}")
    print(f"Test mean RMSE: {s.mean():.3f}")

end_time = time.time()
duration = (end_time - start_time)/60
print(f"ellapsed time: {duration:.2f} minutes")

--- Zone NO1 ---
Train mean RMSE: 14.900
Test mean RMSE: 9.242
--- Zone NO2 ---
Train mean RMSE: 16.335
Test mean RMSE: 10.796
--- Zone NO3 ---
Train mean RMSE: 8.507
Test mean RMSE: 6.060
--- Zone NO4 ---
Train mean RMSE: 5.415
Test mean RMSE: 4.413
--- Zone NO5 ---
Train mean RMSE: 14.010
Test mean RMSE: 7.663
ellapsed time: 49.47 minutes


### 3.2

In [ ]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# global date constants to all zones
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE        = '2023-01-01'
FINAL_DATE_EXPERIMENTS= '2024-12-31'

# features
PRICE_NAMES = ['P_center', 'P_daily_mean']
DA_NAMES   = ["Load_DA", "WindOn_DA"]
FUEL_NAMES = ["Coal","NGas","Oil","EUA"]

# global constants
WD                  = [0]
PRICE_LAGS          = [1,2,7]
DA_LAG              = [0]
FUEL_LAGS           = [2]

# hyperparameters
WINDOW_DAYS   = 730                 
HIDDEN_DIM    = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY  = 0
EPOCHS        = 60
BATCH_SIZE    = 64
ALPHA         = 0.002 # time-decaying weight for MSE loss or X and y

#%cd /content/DLiE_forecast <- Collab
#repo_root  = Path.cwd().resolve() <-Collab
repo_root  = Path.cwd().parents[1]
mapcodes   = ["NO1", "NO2", "NO3", "NO4", "NO5"]
zone_data  = {}          

start_time = time.time()

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df_raw   = pd.read_csv(csv_path, parse_dates=["time_utc"])

    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz      = "CET",
        seed    = 42,
        test_days = (pd.Timestamp(FINAL_DATE_EXPERIMENTS)
                     - pd.Timestamp(INIT_TEST_DATE)).days + 1,
        dtype   = torch.float64,
    )

    idx = pd.DatetimeIndex(sorted(train_dates))
    start_i = idx.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    end_i   = idx.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t  = data_t[start_i:end_i+1]             
    dates_t = pd.Series(train_dates[start_i:end_i+1])

    zone_data[code] = dict(
        df        = df_raw,
        tensor    = data_t,
        dates     = dates_t,
        price_t   = price_t[start_i:end_i+1],
    )

# rolling-window MLP per zone
results_by_zone = {}

for code in mapcodes:
    
    reg_data = build_regression_matrix_reordered_dtleak(
        dat_eval = zone_data[code]["tensor"].cpu().numpy(),
        days_eval= pd.to_datetime(zone_data[code]["dates"]),
        reg_names= zone_data[code]["df"].columns[1:],   
        wd = WD,
        price_lags = PRICE_LAGS,
        da_lag = DA_LAG,
        fuel_lags = FUEL_LAGS,
        da_names = DA_NAMES,
        fuel_names = FUEL_NAMES,
        price_names = PRICE_NAMES
    )
    reg_df   = reg_data["regmat"].dropna().reset_index(drop=True)
    dep_idx  = reg_data["dep_indices"]

    all_cols = reg_df.columns.values
    target_cols = [reg_df.columns[i] for i in dep_idx]
    
    # base_feats = np.unique([
    #     col.rsplit('_s', 1)[0] if '_s' in col else col
    #     for col in all_cols
    # ])
    # base_targets = sorted({
    #     col.rsplit('_s', 1)[0] if '_s' in col else col
    #     for col in target_cols
    # })

    print(f"Features used in {code}: {all_cols}")
    print(f"Target variable in {code}: {target_cols}")
    
    all_dates = pd.DatetimeIndex(sorted(zone_data[code]["dates"]\
                                        .iloc[len(zone_data[code]["dates"])
                                             - len(reg_df):]))
    test_start_row = all_dates.get_loc(pd.Timestamp(INIT_TEST_DATE))
    test_end_row   = all_dates.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    horizon        = test_end_row - test_start_row + 1      

    preds, trues, train_rmses, test_rmses = build_mlp_rolling_forecasts_weighted_data(
        regmat_df   = reg_df.astype("float32"),
        dep_indices = dep_idx,
        window      = WINDOW_DAYS,
        horizon     = horizon,
        start_row   = test_start_row,
        hidden_dim  = HIDDEN_DIM,
        lr          = LEARNING_RATE,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
        epochs      = EPOCHS,
        alpha       = ALPHA,
        device      = device
    )

    results_by_zone[code] = {
        "preds":       preds,
        "trues":       trues,
        "train_rmses": train_rmses,
        "test_rmses":   test_rmses,
    }
  
    # train vs test rmse summary per region
    t = np.array(train_rmses)
    s = np.array(test_rmses)
    print(f"--- Zone {code} ---")
    print(f"Train mean RMSE: {t.mean():.3f}")
    print(f"Test mean RMSE: {s.mean():.3f}")

end_time = time.time()
duration = (end_time - start_time)/60
print(f"ellapsed time: {duration:.2f} minutes")

Features used in NO1: ['P_daily_mean' 'P_center_s0' 'Price_lag_1_s0' 'Price_lag_2_s0'
 'Price_lag_7_s0' 'Load_DA_lag_0_s0' 'WindOn_DA_lag_0_s0' 'P_center_s1'
 'Price_lag_1_s1' 'Price_lag_2_s1' 'Price_lag_7_s1' 'Load_DA_lag_0_s1'
 'WindOn_DA_lag_0_s1' 'P_center_s2' 'Price_lag_1_s2' 'Price_lag_2_s2'
 'Price_lag_7_s2' 'Load_DA_lag_0_s2' 'WindOn_DA_lag_0_s2' 'P_center_s3'
 'Price_lag_1_s3' 'Price_lag_2_s3' 'Price_lag_7_s3' 'Load_DA_lag_0_s3'
 'WindOn_DA_lag_0_s3' 'P_center_s4' 'Price_lag_1_s4' 'Price_lag_2_s4'
 'Price_lag_7_s4' 'Load_DA_lag_0_s4' 'WindOn_DA_lag_0_s4' 'P_center_s5'
 'Price_lag_1_s5' 'Price_lag_2_s5' 'Price_lag_7_s5' 'Load_DA_lag_0_s5'
 'WindOn_DA_lag_0_s5' 'P_center_s6' 'Price_lag_1_s6' 'Price_lag_2_s6'
 'Price_lag_7_s6' 'Load_DA_lag_0_s6' 'WindOn_DA_lag_0_s6' 'P_center_s7'
 'Price_lag_1_s7' 'Price_lag_2_s7' 'Price_lag_7_s7' 'Load_DA_lag_0_s7'
 'WindOn_DA_lag_0_s7' 'P_center_s8' 'Price_lag_1_s8' 'Price_lag_2_s8'
 'Price_lag_7_s8' 'Load_DA_lag_0_s8' 'WindOn_DA_lag_0_s8' 'P_c

In [ ]:
'''
I want to do several feature engineering methods. Note I will not include them at once, only individually to understand their effect to model training:
1) Lag de_meaned_price and use it as a regressor and include (or not necessary? Confirm) P_daily_mean
2) Lag price difference as it is done here: '''

['P_daily_mean',
 'P_center_s0',
 'Price_lag_1_s0',
 'Price_lag_2_s0',
 'Price_lag_7_s0',
 'Load_DA_lag_0_s0',
 'WindOn_DA_lag_0_s0',
 'P_center_s1',
 'Price_lag_1_s1',
 'Price_lag_2_s1',
 'Price_lag_7_s1',
 'Load_DA_lag_0_s1',
 'WindOn_DA_lag_0_s1',
 'P_center_s2',
 'Price_lag_1_s2',
 'Price_lag_2_s2',
 'Price_lag_7_s2',
 'Load_DA_lag_0_s2',
 'WindOn_DA_lag_0_s2',
 'P_center_s3',
 'Price_lag_1_s3',
 'Price_lag_2_s3',
 'Price_lag_7_s3',
 'Load_DA_lag_0_s3',
 'WindOn_DA_lag_0_s3',
 'P_center_s4',
 'Price_lag_1_s4',
 'Price_lag_2_s4',
 'Price_lag_7_s4',
 'Load_DA_lag_0_s4',
 'WindOn_DA_lag_0_s4',
 'P_center_s5',
 'Price_lag_1_s5',
 'Price_lag_2_s5',
 'Price_lag_7_s5',
 'Load_DA_lag_0_s5',
 'WindOn_DA_lag_0_s5',
 'P_center_s6',
 'Price_lag_1_s6',
 'Price_lag_2_s6',
 'Price_lag_7_s6',
 'Load_DA_lag_0_s6',
 'WindOn_DA_lag_0_s6',
 'P_center_s7',
 'Price_lag_1_s7',
 'Price_lag_2_s7',
 'Price_lag_7_s7',
 'Load_DA_lag_0_s7',
 'WindOn_DA_lag_0_s7',
 'P_center_s8',
 'Price_lag_1_s8',
 'Price_lag_